In [1]:
import torch
import torch.nn as nn 
import torch.optim as optim
import numpy as np
from torch.utils.data import random_split
from xgboost.sklearn import XGBClassifier


In [2]:
def load_data(filename):
    data = np.loadtxt(filename, delimiter=',', dtype=np.float32)
    x = data[:, 0:-2]
    y = data[:, [-1]] # after 3 months (-2) and 6 months (-1)
    return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

In [3]:
# target to fix code


class FNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_size)  # 배치 정규화 추가
        )
        self.layer2 = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),  # 추가 은닉층
            nn.ReLU(),
            nn.BatchNorm1d(hidden_size)  # 배치 정규화 추가
        )
        self.layer3 = nn.Sequential(
            nn.Linear(hidden_size, output_size),
            nn.Sigmoid()
        )
        self.dropout = nn.Dropout(0.005)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.dropout(x)
        x = self.layer2(x)
        x = self.dropout(x)  # 추가 은닉층을 위해 드롭아웃 적용
        x = self.layer3(x)
        return x

In [4]:
x_train, y_train = load_data('/Users/withmocha/Library/CloudStorage/OneDrive-SejongUniversity/3학년 2학기/딥러닝개론/과제/과제1/hw1_training_dataset.csv')
x_test, y_test = load_data('/Users/withmocha/Library/CloudStorage/OneDrive-SejongUniversity/3학년 2학기/딥러닝개론/과제/과제1/hw1_test_dataset.csv')

In [5]:
y_train

tensor([[1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
      

In [6]:
x_test

tensor([[0.2474, 0.5331, 0.1690, 0.5000, 0.7500, 0.4545, 0.9561, 0.2500, 0.3926,
         0.2303, 0.2000, 0.0556, 0.0000, 0.3043, 0.0500],
        [0.2474, 0.5331, 0.1690, 0.5000, 0.7500, 0.4545, 0.9561, 0.2500, 0.3926,
         0.2303, 0.3000, 0.0556, 0.0000, 0.3043, 0.0500],
        [0.3268, 0.3718, 0.3380, 0.5000, 0.5000, 0.5455, 0.5575, 0.2500, 0.3998,
         0.2061, 0.7000, 0.6111, 1.0000, 0.1652, 0.4800],
        [0.0042, 0.4179, 0.2535, 0.5000, 0.2500, 0.2727, 0.3986, 0.0000, 0.0000,
         0.2388, 0.6000, 0.5000, 1.0000, 0.1739, 0.7500],
        [0.3268, 0.3718, 0.3380, 0.5000, 0.5000, 0.5455, 0.5575, 0.2500, 0.3998,
         0.2061, 1.0000, 0.7222, 0.0000, 0.1739, 0.0000],
        [0.3208, 0.4467, 0.2676, 0.5000, 0.7500, 0.2727, 0.7489, 0.2812, 0.3559,
         0.1939, 1.0000, 0.7222, 0.0000, 0.3043, 0.7500],
        [0.0084, 0.5389, 0.0000, 1.0000, 0.2500, 0.1818, 0.6514, 0.0000, 0.0735,
         0.3152, 0.4000, 1.0000, 1.0000, 0.2609, 0.0000],
        [0.0168, 0.7176, 0.

In [7]:
x_train.shape

torch.Size([103, 15])

In [8]:


from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test) # test set에는 transform만 사용하기


x_train=torch.Tensor(x_train)
x_test=torch.Tensor(x_test)

In [9]:
model = FNN(input_size=15, hidden_size=300, output_size=1)
criterion = nn.BCELoss()
#optimizer = optim.Adam(model.parameters(),lr=0.05, betas=(0.95, 0.999),eps=1e-08,weight_decay=0.0001) # target to fix
optimizer = optim.Adam(model.parameters(),lr=0.05, betas=(0.95, 0.999),eps=1e-08,weight_decay=0.0001) # target to fix

In [10]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-0.1568, -0.0726, -0.2232,  ...,  0.0844, -0.1112, -0.1387],
        [ 0.1814,  0.0192, -0.1970,  ...,  0.0630,  0.1050, -0.2128],
        [-0.0773,  0.1341,  0.2528,  ..., -0.1986,  0.0163,  0.0339],
        ...,
        [-0.2451,  0.1972,  0.0244,  ..., -0.0982, -0.1687,  0.1949],
        [-0.2042,  0.0797,  0.2302,  ...,  0.2325, -0.1627, -0.1920],
        [-0.1977,  0.2555, -0.0098,  ..., -0.0126,  0.1509, -0.1652]],
       requires_grad=True)
Parameter containing:
tensor([ 0.2521, -0.0132, -0.1192,  0.0295, -0.0128, -0.0062,  0.1215,  0.1757,
        -0.1484,  0.0659, -0.2459, -0.1586, -0.0161,  0.0072,  0.0256,  0.0516,
         0.1169,  0.2292, -0.0928,  0.0512,  0.1915, -0.0099,  0.1053, -0.1512,
         0.0377, -0.1060, -0.0546,  0.1644,  0.0399, -0.0659,  0.0243,  0.1850,
         0.2436, -0.0108,  0.2319,  0.0467, -0.2021,  0.2274, -0.2248,  0.1089,
         0.2007, -0.1554, -0.1254,  0.2553,  0.2399, -0.2193, -0.2487, -0.2522,
         0.1305

In [11]:
# assuming y is torch.Tensor

In [12]:
num_epochs = 1001

for epoch in range(num_epochs):
    outputs = model(x_train)
    loss = criterion(outputs, y_train)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 200 == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}')
        

with torch.no_grad(): 
    train_accuracy = (model(x_train).gt(0.5).float() == y_train).float().mean().item()
    test_accuracy = (model(x_test).gt(0.5).float() == y_test).float().mean().item()
    
    

print(f'\nTrain Accuracy: {train_accuracy:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')
#Train Accuracy: 0.9709 -> start
#Test Accuracy: 0.9000 -> start

Epoch [0/1001], Loss: 0.7730
Epoch [200/1001], Loss: 0.0296
Epoch [400/1001], Loss: 0.0017
Epoch [600/1001], Loss: 0.0853
Epoch [800/1001], Loss: 0.0201
Epoch [1000/1001], Loss: 0.0186

Train Accuracy: 1.0000
Test Accuracy: 0.8000


In [13]:
model(x_test).gt(0.5)

tensor([[False],
        [False],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [False],
        [ True],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [False],
        [False],
        [ True],
        [False]])

In [14]:
model(x_test).gt(0.5).float()

tensor([[0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.]])

In [15]:
y_test

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.]])